In [1]:
from pydantic_settings import BaseSettings

class Environment(BaseSettings):
    openai_api_key: str

env = Environment()

# 参考

- [LangChain Python API Reference](https://python.langchain.com/api_reference/)
  - [langchain-core](https://python.langchain.com/api_reference/core/index.html)
  - [langchain-openai](https://python.langchain.com/api_reference/openai/index.html)

# [Build a Simple LLM Application with LCEL](https://python.langchain.com/docs/tutorials/llm_chain/)

シンプルなLLMアプリケーションを作成する

- リファレンス
  - [ChatOpenAI](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI)
  - [StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)
  - [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# OpenAIのモデルを利用する
model = ChatOpenAI(model="gpt-4", temperature=0)

# LLMの結果をパースして文字列を取得する
parser = StrOutputParser()

# ユーザーに対して翻訳を促すプロンプト
prompt_template = ChatPromptTemplate([
    ("system", "Translate the following into {language}:"),
    ("user", "{text}")
    
])

# パイプを利用して、プロンプト、モデル、パーサーをつなげる
chain = prompt_template | model | parser

# チェーンを実行する
chain.invoke({
    "language": "Japanese",
    "text": "hi"
})

'こんにちは'

# [Build a Chatbot](https://python.langchain.com/docs/tutorials/chatbot/)

モデルは状態を持たないので、会話をしたいときは会話履歴全体をモデルに渡す必要があります。

ここでは [LangGraph](https://langchain-ai.github.io/langgraph/) の memory 機能を使って、会話の履歴を保持します。

## LangGraph

各LLMエージェントのステップなどをグラフ化して状態管理を行うためのツールです。

- [LangGraphの基本的な使い方](https://zenn.dev/pharmax/articles/8796b892eed183)
- [LangGraph](https://langchain-ai.github.io/langgraph/)
  - [MemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.MemorySaver)
  - [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.StateGraph)


In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = ChatOpenAI(model="gpt-4", temperature=0)

# 新しいGraphを定義
workflow = StateGraph(state_schema=MessagesState)

# モデルを呼び出す関数
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# graphにノードを追加
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# メモリ追加
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": "abc123"})

query = "わたしはボブです"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

こんにちは、ボブさん。何かお手伝いできることがありますか？


In [8]:
query = "私の名前は何ですか?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

あなたの名前はボブさんです。
